## The Walks of Nathan Lowell
A look at Nate's path to the back gate.

The journey begins [#tommw](https://www.nathanlowell.com/tommw/)

In [1]:
from io import BytesIO
import json
from PIL import Image
import shutil

import arrow
from bs4 import BeautifulSoup
import requests

import photo as pto

[#tommw](https://www.nathanlowell.com/tommw/) is a cross channel social media campaign architeched and executed by science fiction author Nathan Lowell. 

There are 4 planks to Nathan's campain:
1. A blog hosted at https://www.nathanlowell.com/tommw/
2. A photo stream hosted on [FlickR](https://www.flickr.com/photos/nlowell/)
3. Podcast feed via [RSS](http://www.nathanlowell.com/tommw/feed/)
4. Posts to twitter with the #tommw hastag on [@nlowell on twitter](https://twitter.com/nlowell)

In [2]:
def get_archive():

    # Load the archive file
    try:
        with open('archive.json', 'r') as file:
            archive= json.loads(file.read())
        print('Archive loaded.')
        
    except:
        # Create archive data structure.

        archive = {'meta':{'ids':[],
                          'last_updated':'New File',
                          },
                  'data':[],
                  }
        print('No archived file, building new archive.json')
        
    return archive
# Scrape the blog for links to the other media components
blog_page = requests.get("https://www.nathanlowell.com/tommw/")
soup = BeautifulSoup(blog_page.text, 'html.parser')

# Find a flickr link
flickr = soup.find_all(href=True, attrs={'data-flickr-embed':'true'})

if not flickr:
    print('No flickr links found on blog home page.')
    flicker_link = None
    
else:
    flickr_link = flickr[0]['href']
    
    

In [ ]:
def process_photo_stream_page(stream, archive=None):
    
    if not archive:
        archive = get_archive()
    
    for photo in stream:
        image_name = photo.split('/')[-1]
        photo_id = image_name.split('_')[0]

        if photo_id in archive['meta']['ids']:
            # skip photos already recorded
            continue

        # Save the thumb size image
        thumb_name = 'images/' + photo_id + '.jpg'
        image = requests.get(photo, stream=True)
        
        
        with open(thumb_name, 'wb') as image_file:
            shutil.copyfileobj(image.raw, image_file)

        # Only the original has EXIF data
        info = pto.get_original_photo(photo_id)
        
        # Format the EXIF data
        if info:
            info['photo_id'] = photo_id
            info['thumb'] = thumb_name
            p_url = info.get('link')
            if p_url:
                camera, timestamp = pto.get_exif(p_url)
            else:
                camera, timestamp = ('Error', 'Error' )
            info['camera'] = camera
            info['timestamp'] = timestamp
            archive['data'].append(info)
            archive['meta']['ids'].append(photo_id)
            archive['meta']['last_updated'] = str(arrow.now())
            print(photo_id + ' added')

        else:
            print('no #tommow tag found')
            
    return archive

In [ ]:
# Future Main
archive = get_archive()

if len(archive['meta']['ids']) < 1000:
    archive = pto.build_flickr_archive(flickr_link, archive)
    pass

#stream_url = pto.get_stream_url(flickr_link)
stream = pto.get_photostream(stream_url)
archive = pto.process_photo_stream_page(stream, archive)

No archived file, building new archive.json
https://www.flickr.com/photos/nlowell/page1
32497027918 added
45427063755 added
46313540081 added
45538428704 added
32390140018 added
31253793227 added
46180203671 added
32292571558 added
31207048157 added
46117344171 added
44213546090 added
45104952125 added
32116506418 added
44077082110 added


In [ ]:
with open('archive.json', 'w') as file:
    file.write(json.dumps(archive, indent=4))

In [ ]:
archive

Archive File Structure

``` python
{'id':'123456789','info':{'timestamp':'2018-12-10 10:21', ...}}
```


In [ ]:
archive['meta']['ids'].append('test_id')
archive['meta']['ids']

In [ ]:
test = 'https://www.flickr.com/photos/nlowell/page14'
archive = get_archive()
stream = pto.get_photostream(test)
archive = process_photo_stream_page(stream, archive)

In [ ]:
title = 'this is. a test'
title.find('.')

In [ ]:
if not archive:
    archive = get_archive()

for photo in stream:
    image_name = photo.split('/')[-1]
    photo_id = image_name.split('_')[0]

    if photo_id in archive['meta']['ids']:
        # skip photos already recorded
        continue

    # Save the thumb size image
    thumb_name = 'images/' + photo_id + '.jpg'
    image = requests.get(photo, stream=True)


    with open(thumb_name, 'wb') as image_file:
        shutil.copyfileobj(image.raw, image_file)

    # Only the original has EXIF data
    info = pto.get_original_photo(photo_id)

    # Format the EXIF data
    if info:
        info['photo_id'] = photo_id
        info['thumb'] = thumb_name
        p_url = info.get('link')
        if p_url:
            camera, timestamp = pto.get_exif(p_url)
        else:
            camera, timestamp = ('Error', 'Error' )
        info['camera'] = camera
        info['timestamp'] = timestamp
        archive['data'].append(info)
        archive['meta']['ids'].append(photo_id)
        archive['meta']['last_updated'] = str(arrow.now())
        print(photo_id + ' added')

    else:
        print('no #tommow tag found')

In [ ]:
photo_id